# target : tip

In [1]:
import pymysql.cursors
import pandas as pd

# 불러오기
conn = pymysql.connect(host='localhost', user='zeros', 
                       password='160525', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:
    with conn.cursor() as curs:
        sql = "select * from tips;"
        curs.execute(sql)
        rs = curs.fetchall()
        df = pd.DataFrame(rs)
finally:
    conn.close()

# encoding

In [2]:
df["sex"] = df["sex"].replace({"Female" : 0, "Male" : 1})
df["smoker"] = df["smoker"].replace({"No" : 0, "Yes" : 1})
df["day"] = df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3})
df["time"] = df["time"].replace({"Lunch" : 0, "Dinner" : 1})
df['tip_rate']= df['tip']/df['total_bill']

In [3]:
df

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,0.0,0,3.0,1.0,2,0.059447
1,10.34,1.66,1.0,0,3.0,1.0,3,0.160542
2,21.01,3.50,1.0,0,3.0,1.0,3,0.166587
3,23.68,3.31,1.0,0,3.0,1.0,2,0.139780
4,24.59,3.61,0.0,0,3.0,1.0,4,0.146808
...,...,...,...,...,...,...,...,...
244,16.99,1.01,0.0,0,3.0,1.0,2,0.059447
245,16.99,1.01,1.0,0,2.0,NaN,3,0.059447
246,16.99,1.01,0.0,0,0.0,NaN,3,0.059447
247,16.99,1.01,1.0,0,NaN,1.0,3,0.059447


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           1
smoker        0
day           1
time          2
size          0
tip_rate      0
dtype: int64

# null값 처리

In [5]:
from numpy import isnan
from sklearn.impute import SimpleImputer

In [6]:
# y=타겟
y = df['tip']
X = df.drop('tip',axis=1) #학습할 x에서 tip을 드랍

In [7]:
data = X.values
y = y.values

In [8]:
#NULL값 확인
sum(isnan(data).flatten()), sum(isnan(y).flatten())

(4, 0)

In [9]:
#IMPUTED DATA SET(널없는 데이터)
imputer = SimpleImputer(strategy='median')
imputer.fit(data)

SimpleImputer(strategy='median')

In [10]:
imputer

SimpleImputer(strategy='median')

In [11]:
data_trans = imputer.transform(data) 

In [12]:
data_trans

array([[16.99      ,  0.        ,  0.        , ...,  1.        ,
         2.        ,  0.05944673],
       [10.34      ,  1.        ,  0.        , ...,  1.        ,
         3.        ,  0.16054159],
       [21.01      ,  1.        ,  0.        , ...,  1.        ,
         3.        ,  0.16658734],
       ...,
       [16.99      ,  0.        ,  0.        , ...,  1.        ,
         3.        ,  0.05944673],
       [16.99      ,  1.        ,  0.        , ...,  1.        ,
         3.        ,  0.05944673],
       [16.99      ,  1.        ,  1.        , ...,  0.        ,
         5.        ,  0.05944673]])

In [13]:
df1 = pd.DataFrame(data_trans, columns=['total_bill','sex','smoker','day','time','size','tip_rate'])

In [14]:
df1.isnull().sum()

total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
tip_rate      0
dtype: int64

In [15]:
df1

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
1,10.34,1.0,0.0,3.0,1.0,3.0,0.160542
2,21.01,1.0,0.0,3.0,1.0,3.0,0.166587
3,23.68,1.0,0.0,3.0,1.0,2.0,0.139780
4,24.59,0.0,0.0,3.0,1.0,4.0,0.146808
...,...,...,...,...,...,...,...
244,16.99,0.0,0.0,3.0,1.0,2.0,0.059447
245,16.99,1.0,0.0,2.0,1.0,3.0,0.059447
246,16.99,0.0,0.0,0.0,1.0,3.0,0.059447
247,16.99,1.0,0.0,2.0,1.0,3.0,0.059447


# RFE
- 중요도가 낮은 변수를 제거하는 방식.

In [16]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
# 타겟변수가 뉴메릭 이라서 svr을 사용(suport vector regression)

In [17]:
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=4)

In [18]:
# fit RFE
rfe.fit(df1, y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [19]:
df1.shape

(249, 7)

In [20]:
for i in range(df1.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=False, Rank: 2
Column: 2, Selected=False, Rank: 3
Column: 3, Selected=True, Rank: 1
Column: 4, Selected=True, Rank: 1
Column: 5, Selected=False, Rank: 4
Column: 6, Selected=True, Rank: 1


In [21]:
df1.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'], dtype='object')

In [22]:
# 랭킹에 따라서 랭크1을 제외한 나머지는 숙청
df1.drop(['sex','smoker','size'], axis=1)

,total_bill,day,time,tip_rate
0,16.99,3.0,1.0,0.059447
1,10.34,3.0,1.0,0.160542
2,21.01,3.0,1.0,0.166587
3,23.68,3.0,1.0,0.139780
4,24.59,3.0,1.0,0.146808
...,...,...,...,...
244,16.99,3.0,1.0,0.059447
245,16.99,2.0,1.0,0.059447
246,16.99,0.0,1.0,0.059447
247,16.99,2.0,1.0,0.059447


# PCA : 차원축소
 - 각 객체의 igenvalue 표시

In [59]:
from sklearn.decomposition import PCA

In [60]:
df.shape

(249, 8)

In [62]:
# pca변형을 정의
trans = PCA(n_components=7)

# 데이터변환
X_dim = trans.fit_transform(data_trans)

In [63]:
# 설명력이 높은 순의 데이터 출력
trans.explained_variance_ratio_.round(3)

array([0.968, 0.018, 0.008, 0.003, 0.003, 0.001, 0.   ])

StandardScaler

In [64]:
# 다시 데이터프레임으로 보기
data_pca = pd.DataFrame(X_dim)

In [65]:
data_pca.describe().round(2)

,0,1,2,3,4,5,6
count,249.00,249.00,249.00,249.00,249.00,249.00,249.00
mean,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00
std,8.84,1.21,0.78,0.47,0.46,0.21,0.06
min,-16.71,-1.64,-2.00,-0.85,-0.73,-0.50,-0.12
25%,-6.37,-1.10,-0.44,-0.32,-0.40,-0.14,-0.03
50%,-2.22,-0.40,-0.17,-0.10,-0.06,-0.09,-0.00
75%,4.38,1.49,0.35,0.32,0.36,0.16,0.03
max,31.05,2.52,3.07,1.24,1.00,0.87,0.50


# Regression Feature Selection

In [51]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [32]:
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(df1, y)

print(X_selected.shape)

(249, 4)


In [33]:
df3 = pd.DataFrame(X_selected)

In [34]:
df3

,0,1,2,3
0,16.99,3.0,2.0,0.059447
1,10.34,3.0,3.0,0.160542
2,21.01,3.0,3.0,0.166587
3,23.68,3.0,2.0,0.139780
4,24.59,3.0,4.0,0.146808
...,...,...,...,...
244,16.99,3.0,2.0,0.059447
245,16.99,2.0,3.0,0.059447
246,16.99,0.0,3.0,0.059447
247,16.99,2.0,3.0,0.059447
